selenium:

selenium is an open-source web-based automation tool. Selenium primarily used for testing in the industry but It can also be used for web scraping.

in order to import data from a website with a javascript table hiding the data we needed to use selenium, specifically for its very useful .click() function which allows you to, as the name would suggest,click a link on the webpage, selenium is mostly used for automation however we used this function along with a few others without automation, as the page links were always in the same relative path.

selenium essentially "runs" the webpage on your computer, which allows for automatic, human like scraping, while running it would seem as if a user is slowly browsing the webpage, but it also imports all the data.

read up more on selenium here: https://www.analyticsvidhya.com/blog/2020/08/web-scraping-selenium-with-python/

example of selenium in action:

![SegmentLocal](selenium_example.gif "selenium")

we install relevant drivers..,

In [1]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --user pipenv


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install igdb-api-v4

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install python-oauth2

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install oauthlib

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install selenium


Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import pandas as pd
import time
import requests
from urllib.request import Request,urlopen
from bs4 import BeautifulSoup
import numpy as np
import time
from PIL import Image
import io
import json
from pandas.io.json import json_normalize
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.by import By
from oauthlib.oauth2 import WebApplicationClient
from lxml import etree
from lxml import html
from igdb.wrapper import IGDBWrapper
from igdb.igdbapi_pb2 import GameResult

in order to load data from games database, we had to use selenium, which is a useful tool for crawling as it knows how to "latch" onto a website which uses javascript functions to make it's data harder to extract, it does so by emulating a person pressing the javascript buttons for next page, allowing us to view different pages via code that would not be accessible with beautiful soup, conviently however, selenium is quite compatible with soup and in order to turn a seleium drive object into a soup object we used 2 simple lines of code

In [6]:
def gamesdb_load(year):
    driver = webdriver.Chrome(ChromeDriverManager().install()) #creating a driver object
    search_url= "https://www.gamesdatabase.org/list.aspx?year={}".format(year)
    driver.get(search_url) #using selenium to get the page

    megadf = pd.DataFrame()

    for i in range(2,1000):#i only serves as a convient counter here, the loop gets broken out of once the end is reached
        page_source = driver.page_source #page source is a html objectt now
        soup = BeautifulSoup(page_source, 'lxml') #making it very simple to interact with BS!
        
        headers = [th.getText() for th in soup.findAll('tr',attrs={"style":"color:Black;background-color:#E0E0E0;font-size:Medium;","align":"left"} ,limit=1)[0].findAll('th')]
        headers = headers[1:]
        #extracting df columns from data
        while(len(headers) !=6):
            headers.remove('\xa0')#removing corrupted headers created by the website's junky code
        
        rows = soup.findAll('tr',attrs = {"style":"font-size:Medium;","align":"left"})[1:] #getting all the data from the table on the page
        game_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]#and extracting only the text
        for l in range(4):
            game_stats = np.delete(game_stats, l, 1)#again removing corrupted lines for the same reason as earlier
        
        #creating a df using headers as col names and game stats as rows
        game_stats_df = pd.DataFrame(game_stats, columns = headers) 
        game_stats_df.dropna(subset = ['Game'], inplace = True)
        megadf=pd.concat([megadf,game_stats_df], ignore_index=True)
        #moving on to the next page of the table with selenium
        #we do this beacuse the table is encrypted into a single page rather than spanning over multiple links
        if(i % 30 == 1):#specific actions must be taken every 30 pages due to the website's design...
            try:#try this to see if there even is a next page
                driver.find_element_by_xpath("//td/a[text()='...'][last()]")
            except:
                break#there isnt, leave
            else:
                driver.find_element_by_xpath("//td/a[text()='...'][last()]").click() #go to the next 30 pages
                
        #the table on the website is divided to slices of 30 pages, where you gotta manually press to go to the next slice
        #this is impossible to do with beautiful soup, so we use seleium .click() function
        else:#general course of action
            text = "//td/a[text()='{}']".format(i) #finding the path to the button
            try:
                driver.find_elements_by_xpath(text)[0] 
            except:
                break
            else:
                driver.find_elements_by_xpath(text)[0].click() # go to the next page
    #finally we return the df
    return megadf 

In [29]:
FinalDF= DataFrame #simple for loop used to install df
for i in range(1983,2001):
    tempdf = gamesdb_load(i)
    FinalDF=pd.concat([FinalDF,tempdf], ignore_index=True)
FinalDF



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\tal\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
<ipython-input-6-ff34a56ea28e>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install()) #creating a driver object
<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0].click()
<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34

<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0].click()
<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0].click()
<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use

<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0].click()
<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0].click()
<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]


====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version 

<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0].click()
<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0].click()
<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use

<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0].click()
<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0].click()
<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use

...

<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0].click()
<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0].click()
<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use

<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0].click()
<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0].click()
<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use

<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0].click()
<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0].click()
<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use

<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0].click()
<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0].click()
<ipython-input-6-ff34a56ea28e>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath(text)[0]
<ipython-input-6-ff34a56ea28e>:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use

,Game,System,Publisher,Developer,Category,Year
0,1984: The Game of Government Management,Sinclair ZX Spectrum,Incentive Software,Incentive Software,Strategy,1983
1,1994: Ten Years After,Sinclair ZX Spectrum,Visions (Software Factor...,Visions (Software Factor...,Action,1983
2,1994: Ten Years After,Commodore 64,Visions (Software Factor...,Visions (Software Factor...,Action,1983
3,2D Narrow Escape,GCE Vectrex,GCE,Fred Taft,,1983
4,3D Bowling,Emerson Arcadia 2001,UA Ltd,,Sports,1983
...,...,...,...,...,...,...
23964,Xtreme Sports,Sega Dreamcast,Atari,InnerLoop,Sports,2000
23965,Yukyu Gensokyoku: Perpetual Collection,Sega Saturn,Media Works,Media Works,Strategy,2000
23966,Zelda no Densetsu: Mujura no Kamen,Nintendo N64,Nintendo,Nintendo,Role-Playing,2000
23967,Zoku Otenamihaiken,Arcade,Success - ASCII,Success Corp.,,2000


In [134]:
FinalDF.to_pickle('gamesDB.pkl') #saving it

IGDB is one of the world's biggest video games databases, and it allows for users to access its api via requests.
in order to get those requests in python we used IGDB Python API V4 which is a library specifically designed to import data from igdb, it uses a wrapper which creates a code that we imported from the website's api section.

to read more: https://github.com/twitchtv/igdb-api-python



In [5]:
def year_taker_igdb(year):
    #creating wrapper for more convienient data importation
    wrapper = IGDBWrapper("07l3cysdx76p4wm8b2q82ubexm0vp2", "vnfmyo1fcmhczwi37qnxurhsxhc2wd") #the variables are codes we got from igdb api page
    finaldf= pd.DataFrame()
    byte_array = wrapper.api_request( 
                'games', 
                'fields name,total_rating,game_modes,rating,artworks,involved_companies.company.name,keywords.name,videos,platforms,follows,genres,release_dates.m; where release_dates.y = {};'.format(year))
    #wrapper import request form, 'games' is the request path for the IGDB data about video games
    #fields are the data we ask from the api with our request.
    #where is the requirement for specific games, such as games from the year 2007.\
    print(year)
    num = -59
    for i in range(40):
        #For loop exists so we could only extract some of the data, as we do not require all 3,000,000+ video games.
        #had we wanted to, we couldve instead used a while loop to get all of the data.
        g = json.loads(byte_array) #translation of a byte_array type into a dict
        
        for item in g:
            #some items dont have release dates, this is done to prevent errors in such a case
            try:
                item['release_dates']
            except KeyError:
                item['month'] = ''
            else:
                if('m' in item['release_dates'][0]):
                    item['month']= item['release_dates'][0]['m']
                else:
                    item['month']= np.NAN
                item.pop('release_dates')
                
            #only taking the 3 most relevant commpanies if they are known
            try:
                item['involved_companies']
            except KeyError:
                relevant_companies = ['']
            else:
                if(len(item['involved_companies'])>=3):
                    relevant_companies = [item['involved_companies'][0]['company']['name'],item['involved_companies'][1]['company']['name'],item['involved_companies'][2]['company']['name']]
                elif(len(item['involved_companies'])==2):
                    relevant_companies = [item['involved_companies'][0]['company']['name'],item['involved_companies'][1]['company']['name']]
                elif(len(item['involved_companies'])==1):
                    relevant_companies = [item['involved_companies'][0]['company']['name']]
                item.pop('involved_companies')

            item['companies'] = relevant_companies
            item['year'] = year
            
            #creating df from dict and insertingg it to the final df
            tempdf = pd.DataFrame.from_dict(item,orient = 'index')
            tempdf = tempdf.transpose()
            finaldf=pd.concat([finaldf,tempdf], ignore_index=True)
        #wrapper requests only return 10 games each time
        #in order to get the next 10 games, we must make a new request
        if num == finaldf['id'].iloc[-1]:#however, we break early if we've already extracted all of the data before i = 40
            print("leaving early at i = ",i)
            break
        num=finaldf['id'].iloc[-1]
        #we limit id to be higher than the highest id currently in the df
        #thats beacuse api request return the smallest id values possible, in ascending order, always.
        byte_array = wrapper.api_request(
                    'games',
                    'fields name,game_modes,rating,status,total_rating,artworks,involved_companies.company.name,videos,keywords.name,platforms,follows,genres,release_dates.m; where release_dates.y = {} & id > {};'.format(year,num))
        
    print()
    return finaldf #finally, we return the completed df


In [6]:
IGDB_DF_OLD = pd.DataFrame() 
for i in range(1980,2001):
    tempdf = year_taker_igdb(i)
    IGDB_DF_OLD = pd.concat([IGDB_DF_OLD,tempdf], ignore_index=True)
IGDB_DF_OLD

1980
leaving early at i =  20

1981
leaving early at i =  35

1982

1983

1984

1985

1986

1987

1988

1989

1990

1991

1992

1993

1994

1995

1996

1997

1998

1999

2000



,id,game_modes,genres,name,platforms,rating,total_rating,month,companies,year,artworks,follows,keywords,videos,status
0,651,[1],[31],The Wizard and the Princess,"[13, 15, 65, 75, 125, 149, 152]",58.457823,58.457823,12,"[IBM, On-Line Systems, Melbourne House]",1980,NaN,NaN,NaN,NaN,NaN
1,1944,[1],[31],Zork I: The Great Underground Empire,"[13, 14, 15, 16, 25, 63, 65, 75, 93, 94, 126, ...",68.860738,68.860738,12,"[Infocom, Commodore Electronics Ltd., SystemSoft]",1980,[40859],6,"[{'id': 218, 'name': 'interactive fiction'}, {...",[23929],NaN
2,2298,[1],[31],Adventureland,"[15, 26, 65, 69, 71, 75, 90, 125, 126, 129, 13...",NaN,NaN,12,"[Creative Computing Software, Texas Instrument...",1980,NaN,NaN,"[{'id': 151, 'name': 'medieval'}, {'id': 206, ...",[45636],NaN
3,2391,[1],"[5, 13]",Berzerk,"[52, 59, 66, 70]",70.197365,70.098682,12,"[Stern Electronics, Stern Electronics, Atari, ...",1980,NaN,1,"[{'id': 67, 'name': 'robot'}, {'id': 575, 'nam...",[15478],NaN
4,2417,[1],[12],Akalabeth: World of Doom,"[3, 6, 13, 14, 39, 75]",51.0246,51.0246,12,"[California Pacific Computer, Quantum Toast, R...",1980,NaN,6,"[{'id': 296, 'name': 'fantasy'}, {'id': 324, '...",[23919],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8136,9034,"[1, 2]","[11, 15, 16, 24]",Battle Isle: The Andosia War,[6],NaN,NaN,11,"[Cauldron HQ, Blue Byte Software]",2000,NaN,NaN,"[{'id': 70, 'name': 'futuristic warfare'}, {'i...",NaN,NaN
8137,9060,"[1, 2]",[5],Laser Arena,[6],NaN,NaN,9,"[Trainwreck Studios, ValuSoft]",2000,NaN,1,"[{'id': 114, 'name': 'arena'}, {'id': 453, 'na...",[31756],NaN
8138,9067,"[1, 2, 3]","[11, 15]",StarCraft 64,[4],86.596368,86.596368,6,"[Mass Media Inc, Nintendo, Blizzard Entertainm...",2000,NaN,NaN,"[{'id': 526, 'name': 'rts'}, {'id': 805, 'name...","[39386, 39387]",NaN
8139,9091,[1],"[2, 31]",Dracula: Resurrection,"[6, 7, 14, 34, 39]",50.469255,50.469255,6,"[Microïds, Anuman Interactive, Index+]",2000,[22484],5,"[{'id': 101, 'name': 'casual'}, {'id': 133, 'n...",[30125],NaN


In [130]:
IGDB_DF_NEW.to_pickle('IGDB_NEW.pkl')

In [11]:
IGDB_DF_OLD.to_pickle('igdb_old.pkl')

this is the function we used to make some dictionaries we used later on..

In [25]:
def make_genric_dict(year):
    #creating wrapper for more convienient data importation
    wrapper = IGDBWrapper("07l3cysdx76p4wm8b2q82ubexm0vp2", "vnfmyo1fcmhczwi37qnxurhsxhc2wd")
    finaldict = {}
    byte_array = wrapper.api_request(
                'games',
                'fields platforms.name; where release_dates.y = {};'.format(year))
    #wrapper import request form, 'games' is the request path for the IGDB data about video games
    #fields are the data we ask from the api with our request.
    #where is the requirement for specific games, such as games from the year 2007.\
    for i in range(15):
        #For loop exists so we could only extract some of the data, as we do not require all 3,000,000+ video games.
        #had we wanted to, we couldve instead used a while loop to get all of the data.
        g = json.loads(byte_array) #translation of a byte_array type into a dict
        for item in g:
            try:
                for i in item['platforms']:
                    if(i['id'] not in finaldict.keys()):
                        finaldict[i['id']] = i['name']
            except:
                pass
            num = item['id']
        byte_array = wrapper.api_request(
                    'games',
                    'fields platforms.name; where release_dates.y = {} & id > {};'.format(year,num))
        
    
    return finaldict #finally, we return the completed df
